In [3]:
from ase.build import bulk
from pyiron_workflow_atomistics.dataclass_storage import CalcInputMinimize
from pyiron_workflow_atomistics.bulk import optimise_cubic_lattice_parameter
from pyiron_workflow_lammps.engine import LammpsEngine
from pyiron_workflow import Workflow

import os

from pyiron_workflow_atomistics.bulk_defect.vacancy import get_vacancy_formation_energy

%load_ext autoreload
%autoreload 2

In [4]:
inp = CalcInputMinimize()
inp.relax_cell = False
Engine = LammpsEngine(EngineInput = inp)

Engine.working_directory = "test_opt_cubic_cell"
# Adjust this line to match your LAMMPS installation
# If you are using conda-lammps or a LAMMPS binary on your system, you can use directly:
Engine.command = "lmp -in in.lmp -log minimize.log"
Engine.lammps_log_filepath = "minimize.log"
Engine.input_script_pair_style = "eam/fs"
import os
Engine.path_to_model = os.getcwd() + "/Al-Fe.eam.fs"
# /root/github_dev/pyiron_workflow_atomistics/pyiron_workflow_atomistics
structure = bulk("Fe", a=2.828, cubic=True)
wf1 = Workflow(Engine.working_directory, delete_existing_savefiles=True)
wf1.opt_cubic_cell = optimise_cubic_lattice_parameter(
    structure=structure,
    name="Fe",
    crystalstructure="bcc",
    calculation_engine=Engine,
    parent_working_directory="opt_cubic_cell",
    rattle=0.1,
    strain_range=(-0.02, 0.02),
    num_points=6,
    eos_type="birchmurnaghan",
)
# Any eos_type supported by ase.eos.EquationOfState can be used,
# e.g. sjeos, taylor, murnaghan, birch, birchmurnaghan, pouriertarantola, vinet, antonschmidt, p3

wf1.run()


/root/miniconda3/envs/pyiron_pypi_workflow_lammps/lib/python3.12/site-packages/pyiron_workflow_lammps/engine.py:113: RuntimeWarning: 'mode' already set to 'minimize' and also implied by EngineInput; no change.
  warnings.warn(
2025-09-25 02:08:29,444 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell/strain_000' already exists. Existing files may be overwritten.


current mode  minimize


2025-09-25 02:08:31,548 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell/strain_001' already exists. Existing files may be overwritten.
2025-09-25 02:08:33,230 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell/strain_002' already exists. Existing files may be overwritten.
2025-09-25 02:08:34,945 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell/strain_003' already exists. Existing files may be overwritten.
2025-09-25 02:08:36,579 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell/strain_004' already exists. Existing files may be overwritten.
2025-09-25 02:08:38,271 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell/strain_005' already exists. Existing files may be overwritten.


{'opt_cubic_cell__equil_struct': Atoms(symbols='Fe2', pbc=True, cell=[2.8554036006730863, 2.8554036006730863, 2.8554036006730863], initial_magmoms=...),
 'opt_cubic_cell__a0': np.float64(2.8554036006730863),
 'opt_cubic_cell__B': np.float64(178.2689296133779),
 'opt_cubic_cell__equil_energy_per_atom': np.float64(-4.012989682254272),
 'opt_cubic_cell__equil_volume_per_atom': np.float64(11.640523523888817),
 'opt_cubic_cell__volumes': [np.float64(21.287097162601995),
  np.float64(21.81268105404494),
  np.float64(22.346845717919777),
  np.float64(22.889660634165235),
  np.float64(23.441195282719992),
  np.float64(24.001519143522813)],
 'opt_cubic_cell__structures': [Atoms(symbols='Fe2', pbc=True, cell=[[2.77144, 1.6970175625144705e-16, 1.6970175625144705e-16], [0.0, 2.77144, 1.6970175625144705e-16], [0.0, 0.0, 2.77144]]),
  Atoms(symbols='Fe2', pbc=True, cell=[[2.794064, 1.7108707671064253e-16, 1.7108707671064253e-16], [0.0, 2.794064, 1.7108707671064253e-16], [0.0, 0.0, 2.794064]]),
  Ato

# Example for LAMMPS engine

In [ ]:
from pyiron_workflow import Workflow
from pyiron_workflow_atomistics.surface.utils import (
    create_surface_from_symbol,
    create_surface_slab,
    get_surface_info
)
from pyiron_workflow_atomistics.surface.utils import (
    create_bulk_from_symbol)
# supercell import
from pyiron_workflow_atomistics.structure_manipulator.tools import (
        create_supercell_with_min_dimensions,
    )
# Create workflow
wf = Workflow("surface_study")

# Method 1: Direct surface creation from symbol
# wf.fe_slab = create_surface_from_symbol(
#     "Fe", 
#     miller_indices=(1, 1, 0), 
#     layers=4, 
#     vacuum=15.0,
#     crystalstructure="bcc",
#     a=2.87
# )

# Method 2: Create bulk first, then surface
wf.fe_bulk = create_bulk_from_symbol("Fe", crystalstructure="bcc", a=wf1.opt_cubic_cell.outputs.a0.value, cubic=True)
wf.fe_slab = create_surface_slab(
    wf.fe_bulk.outputs.bulk_from_symbol,
    miller_indices=(1, 1, 0), 
    layers=40, 
    vacuum=15.0
)
wf.fe_slab_no_vac = create_surface_slab(
    wf.fe_bulk.outputs.bulk_from_symbol,
    miller_indices=(1, 1, 0), 
    layers=40, 
    vacuum=0.0
)
from pyiron_workflow_atomistics.calculator import calculate_structure_node
wf.calc_slab = calculate_structure_node(wf.fe_slab.outputs.surface_slab, 
                                        calculation_engine=Engine)
wf.calc_slab_no_vac = calculate_structure_node(wf.fe_slab_no_vac.outputs.surface_slab, 
                                        calculation_engine=Engine)
import pyiron_workflow as pwf
@pwf.as_function_node("surface_energy")
def get_surface_energy(slab_calc, slab_calc_no_vac, area):
    energy = (slab_calc - slab_calc_no_vac) / area
    return energy

# Get area of the slab
@pwf.as_function_node("surface_area")
def get_surface_area(slab):
    area = slab.get_volume() / (slab.get_volume() / slab.cell[-1][-1])
    return area
wf.area = get_surface_area(wf.fe_slab.outputs.surface_slab)
wf.surface_energy = get_surface_energy(wf.calc_slab.outputs.calc_output.final_energy,
                                      wf.calc_slab_no_vac.outputs.calc_output.final_energy,
                                      wf.area.outputs.surface_area)
# Run workflow
wf.run()

# Get results
fe_slab = wf.fe_slab.outputs.surface_slab.value

print(f"Fe(110) slab: {len(fe_slab)} atoms")

# Get surface information
wf.surface_info = get_surface_info(fe_slab)
wf.run()
info = wf.surface_info.outputs.surface_info.value
print(f"slab length: {wf.fe_slab.outputs.surface_slab.value.cell[-1][-1]} Å")
print(f"Surface area: {info['surface_area']:.2f} Å²")
print(f"Surface energy: {wf.surface_energy.outputs.surface_energy.value:.3f} eV/Å² or {wf.surface_energy.outputs.surface_energy.value * 16.021766208:.2f} J/m²")

2025-09-25 02:49:01,249 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell' already exists. Existing files may be overwritten.
2025-09-25 02:49:02,881 - pyiron_log - WARNING - Directory 'test_opt_cubic_cell' already exists. Existing files may be overwritten.


Fe(110) slab: 80 atoms
slab length: 108.74393471335655 Å
Surface area: 11.53 Å²
Surface energy: 0.016 eV/Å² or 0.26 J/m²
